In [ ]:
%%bash
python -m spacy download en
python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=1e5107d6dc3eed91c1adfa64aa8d22fef45d5700911d69da7d698796798a7e00
  Stored in directory: /tmp/pip-ephem-wheel-cache-jnzkxbwd/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field , BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 1.03MB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 276kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 266kB/s]


In [ ]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [ ]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7855
Unique tokens in target (en) vocabulary: 5893


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [ ]:
class Encoder(nn.Module):
  def __init__(self , input_dim,emb_dim , enc_hid_dim ,dec_hid_dim, dropout):
    super().__init__()
    self.enc_hid_dim = enc_hid_dim
    self.dec_hid_dim = dec_hid_dim
    self.emb_dim = emb_dim
    self.embedding = nn.Embedding(input_dim,emb_dim)
    self.rnn = nn.GRU(emb_dim ,enc_hid_dim , bidirectional = True)
    self.fc = nn.Linear(enc_hid_dim*2 , dec_hid_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, src):

    embedding = self.dropout(self.embedding(src))
    # Input Dimension : SRC --> [sequence_length , batch_size]
    # Output Dimension : embedding --> [sequence_length , batch_size , embedding dimension]

    outputs , hidden = self.rnn(embedding)
    # Outputs Dimension : outputs --> [sequence_length , batch_size , enc_hid_dim * n directions ]
    # Hidden Dimension :  hidden --> [  n layers * n directions ,batch_size , enc_hid_dim ]

    hidden  = torch.tanh(self.fc(torch.cat((hidden[-2 ,:,:] , hidden[-1 ,:,:]), dim = 1)))
    # hidden[-2 :, : ] --> [batch_size , enc_hid_dim ]
    # hidden[-1 :, : ] --> [batch_size , enc_hid_dim ]
    # Input to Fully connected layer --> torch.cat((hidden[-2 :, : ] , hidden[-1 :, : ]), dim = 1) --> [batch_size , enc_hid_dim * 2]
    # hidden  --> [batch_size , dec_hid_dim ]

    return (outputs , hidden)


In [ ]:
class Attention(nn.Module):

  def __init__(self,enc_hid_dim,dec_hid_dim):
    super().__init__()
    self.attn = nn.Linear(enc_hid_dim * 2 + dec_hid_dim , dec_hid_dim)
    self.v = nn.Linear(dec_hid_dim , 1 , bias = False)

  def forward(self, hidden , encoder_outputs): # hidden is Decoder's hidden 
    src_len = encoder_outputs.shape[0]

    hidden = hidden.unsqueeze(1).repeat(1,src_len,1) # hidden is Decoder's hidden basically S_t-1
    # Input --> [batch_size , dec_hid_dim]
    # After unsqueeze --> [batch_size , 1 ,  dec_hid_dim]
    # After repeat --> [batch_size , sequence_length ,  dec_hid_dim]
    
    encoder_outputs = encoder_outputs.permute(1,0,2)
    # encoder_outputs --> [sequence_length , batch_size , enc_hid_dim * n directions ]
    # encoder_outputs --> (after permute) : [batch_size , sequence_length , enc_hid_dim * n directions ]

    energy = torch.tanh(self.attn(torch.cat((hidden , encoder_outputs ) , dim = 2))) # Basically the similarity of decoder hidden vector(repeated) with encoder hidden vectors
    # energy --> [batch_size , sequence_length ,  dec_hid_dim]

    attention = self.v(energy).squeeze(2)
    # attention --> [batch_size , sequence_length ,  1]
    # attention --> (after squeeze)  [batch_size , sequence_length ] 
    # we want our attention vector to be as long as our input sequence

    return(F.softmax(attention , dim = 1)) 



In [ ]:
class Decoder(nn.Module):
  def __init__(self , output_dim,emb_dim , enc_hid_dim ,dec_hid_dim , dropout , attention):
    super().__init__()
    self.output_dim = output_dim
    self.attention = attention
    self.embedding = nn.Embedding(output_dim,emb_dim)
    self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim , dec_hid_dim)
    self.fc_out = nn.Linear((enc_hid_dim * 2) + emb_dim  + dec_hid_dim , output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self , input , hidden , encoder_outputs):
    #input --> [batch size]
    input = input.unsqueeze(0)
    #input --> [1 , batch size]

    embedded = self.dropout(self.embedding(input))
    # embedding --> [1 , batch size , emb_dim ]

    a = self.attention(hidden , encoder_outputs) # hidden is Decoder's hidden 
    # a -->  [batch_size , sequence_length ] 

    a = a.unsqueeze(1)
    # a -->  [ batch_size , 1 , sequence_length ]

    # encoder_outputs --> [sequence_length , batch_size , enc_hid_dim * 2 ] # ndirections = 2
    encoder_outputs = encoder_outputs.permute(1,0,2)
    # encoder_outputs --> (after permute) : [batch_size , sequence_length , enc_hid_dim * 2 ]

    weighted = torch.bmm(a , encoder_outputs)
    # weighted --> [ batch_size , 1 , enc_hid_dim * 2]

    weighted = weighted.permute(1,0,2)
    # weighted --> [ 1 , batch_size , enc_hid_dim * 2]

    rnn_input = torch.cat((embedded , weighted), dim = 2)
    # rnn_input --> [ 1 , batch_size , enc_hid_dim * 2 + emb_dim ]

    output , hidden = self.rnn(rnn_input , hidden.unsqueeze(0)) # hidden  --> [batch_size , dec_hid_dim ]

    embedded = embedded.squeeze(0)
    output = output.squeeze(0)
    weighted = weighted.squeeze(0)

    prediction = self.fc_out(torch.cat((output , weighted , embedded), dim = 1))

    return prediction , hidden.squeeze(0)



In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM , DEC_HID_DIM) 
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM , ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT,attn)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,518,917 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 19s
	Train Loss: 4.426 | Train PPL:  83.615
	 Val. Loss: 4.015 |  Val. PPL:  55.413
Epoch: 02 | Time: 1m 23s
	Train Loss: 3.154 | Train PPL:  23.424
	 Val. Loss: 3.517 |  Val. PPL:  33.687
Epoch: 03 | Time: 1m 25s
	Train Loss: 2.624 | Train PPL:  13.791
	 Val. Loss: 3.221 |  Val. PPL:  25.053
Epoch: 04 | Time: 1m 27s
	Train Loss: 2.280 | Train PPL:   9.774
	 Val. Loss: 3.125 |  Val. PPL:  22.756
Epoch: 05 | Time: 1m 27s
	Train Loss: 2.015 | Train PPL:   7.499
	 Val. Loss: 3.154 |  Val. PPL:  23.421
Epoch: 06 | Time: 1m 29s
	Train Loss: 1.825 | Train PPL:   6.204
	 Val. Loss: 3.125 |  Val. PPL:  22.770
Epoch: 07 | Time: 1m 28s
	Train Loss: 1.680 | Train PPL:   5.368
	 Val. Loss: 3.086 |  Val. PPL:  21.892
Epoch: 08 | Time: 1m 28s
	Train Loss: 1.535 | Train PPL:   4.643
	 Val. Loss: 3.160 |  Val. PPL:  23.564
Epoch: 09 | Time: 1m 29s
	Train Loss: 1.421 | Train PPL:   4.142
	 Val. Loss: 3.184 |  Val. PPL:  24.145
Epoch: 10 | Time: 1m 28s
	Train Loss: 1.343 | Train PPL

In [ ]:
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM , ENC_DROPOUT)


In [ ]:
outputs , hidden = enc.forward(torch.zeros([12,32]).to(torch.int64))

In [ ]:
outputs.size() , hidden.size()

(torch.Size([12, 32, 1024]), torch.Size([32, 512]))

In [ ]:
s_hidden = hidden

In [ ]:
s_hidden.unsqueeze(1).repeat(1,12,1).size()

torch.Size([32, 12, 512])

In [ ]:
outputs.permute(1,0,2).size()

torch.Size([32, 12, 1024])

In [ ]:
a = torch.zeros([1, 128, 256])
b = torch.zeros([1, 128, 512])
c = torch.zeros([1, 128, 1024])
torch.cat((a , b ,c), dim = 2)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])